# AHP - ANALYTIC HIERARCHY PROCESS

In [1]:
import pandas as pd
import numpy as np

In [2]:
Criterias = ["Style","Reliability","Fuel Economy"]

# Inıtalize the Criteria Matrix

In [3]:
criteria_df = pd.DataFrame([],index = Criterias, columns = Criterias)
criteria_df.iloc[0,0] = 1.0
criteria_df.iloc[1,1] = 1.0
criteria_df.iloc[2,2] = 1.0
criteria_df.iloc[0,1] = 1/2
criteria_df.iloc[0,2] = 3.0
criteria_df.iloc[1,0] = 2.0
criteria_df.iloc[1,2] = 4.0
criteria_df.iloc[2,0] = 1/3
criteria_df.iloc[2,1] = 1/4
criteria_df

,Style,Reliability,Fuel Economy
Style,1.0,0.5,3.0
Reliability,2.0,1.0,4.0
Fuel Economy,0.333333,0.25,1.0


# Get the Eigenvector of the Matrix

In [4]:
#This Method returns the Eigenvector of the given Matrix
def Eigenvector(matrix):
    matrix_power_2 = np.linalg.matrix_power(matrix,2)
    row_sums = matrix_power_2.sum(axis = 1,keepdims = True,dtype = float)
    divider = row_sums.sum(axis = 0,keepdims = True,dtype = float)
    return row_sums/divider,matrix_power_2

In [5]:
# This method returns the difference between the old and the new Eigenvector. Recursively keeps going until their differences reach a VERY small value
def Process(old_Vector,old_Matrix):
        new_Vector,new_Matrix = Eigenvector(old_Matrix)
        difference = new_Vector-old_Vector
        difference = np.asarray(difference,dtype = float)
        if( (difference.sum(axis = 0)) < 0.00000000005 ):
            return new_Vector
        else:
            Process(new_Vector,new_Matrix)

# Importance of criteria

In [6]:
vector,matrix = Eigenvector(criteria_df)
new_Vector = Process(vector,matrix).round(4)
criteria_rank_df = pd.DataFrame(new_Vector,index = Criterias)
criteria_rank_df

,0
Style,0.3196
Reliability,0.5584
Fuel Economy,0.1220


# Now We Can Move On To Our Alternatives

PairWise Comparasions

In [7]:
Alternatives = ["Civic","Saturn","Escort","Clio"]

style_df = pd.DataFrame([],index = Alternatives,columns = Alternatives)
style_df.iloc[0,0] = 1
style_df.iloc[0,1] = 1/4
style_df.iloc[0,2] = 4
style_df.iloc[0,3] = 1/6

style_df.iloc[1,0] = 4
style_df.iloc[1,1] = 1
style_df.iloc[1,2] = 4
style_df.iloc[1,3] = 1/4

style_df.iloc[2,0] = 1/4
style_df.iloc[2,1] = 1/4
style_df.iloc[2,2] = 1
style_df.iloc[2,3] = 1/5

style_df.iloc[3,0] = 6
style_df.iloc[3,1] = 4
style_df.iloc[3,2] = 5
style_df.iloc[3,3] = 1
style_df

,Civic,Saturn,Escort,Clio
Civic,1,0.25,4,0.166667
Saturn,4,1,4,0.25
Escort,0.25,0.25,1,0.2
Clio,6,4,5,1


In [8]:
relaibility_df = pd.DataFrame([],index = Alternatives,columns = Alternatives)

relaibility_df.iloc[0,0] = 1
relaibility_df.iloc[0,1] = 2
relaibility_df.iloc[0,2] = 5
relaibility_df.iloc[0,3] = 1

relaibility_df.iloc[1,0] = 1/2
relaibility_df.iloc[1,1] = 1
relaibility_df.iloc[1,2] = 3
relaibility_df.iloc[1,3] = 2

relaibility_df.iloc[2,0] = 1/5
relaibility_df.iloc[2,1] = 1/3
relaibility_df.iloc[2,2] = 1
relaibility_df.iloc[2,3] = 1/4

relaibility_df.iloc[3,0] = 1
relaibility_df.iloc[3,1] = 1/2
relaibility_df.iloc[3,2] = 4
relaibility_df.iloc[3,3] = 1

relaibility_df

,Civic,Saturn,Escort,Clio
Civic,1,2,5,1
Saturn,0.5,1,3,2
Escort,0.2,0.333333,1,0.25
Clio,1,0.5,4,1


The EigenValues of these pairwase comparisons will give the rank of the alternative in the criteria

In [9]:
temp_style,temp_matrix = Eigenvector(style_df)
rank_style = Process(temp_style,temp_matrix)
rank_style = rank_style.round(4) 
style_rank = pd.DataFrame(rank_style, index = Alternatives, columns = ["Style Rank"])
style_rank

,Style Rank
Civic,0.1168
Saturn,0.2458
Escort,0.0607
Clio,0.5766


In [10]:
temp_rely, rely_matrix = Eigenvector(relaibility_df)
rank_rely = Process(temp_rely,rely_matrix)
rank_rely = rank_rely.round(4)
rely_rank = pd.DataFrame(rank_rely, index = Alternatives, columns = ["Reliability Rank"])
rely_rank

,Reliability Rank
Civic,0.3788
Saturn,0.2900
Escort,0.0742
Clio,0.2570


In [11]:
fuel_df = pd.DataFrame([], index = Alternatives)
fuel_df["Civic"] = 34
fuel_df["Saturn"] = 27
fuel_df["Escort"] = 24
fuel_df["Clio"] = 28
fuel_df = pd.DataFrame(fuel_df.iloc[0,:])
fuel_df.rename(columns = {'Civic':'Fuel Economy'}, inplace = True)
fuel_df

,Fuel Economy
Civic,34
Saturn,27
Escort,24
Clio,28


To use Fuel we have to normalize its values

In [12]:
sum = fuel_df.sum(axis = 0)
normal_fuel_df = fuel_df/sum
normal_fuel_df

,Fuel Economy
Civic,0.300885
Saturn,0.238938
Escort,0.212389
Clio,0.247788


Now we have pairwase comparions for each Criteria 

In [13]:
normal_fuel_df["Style"] = style_rank
normal_fuel_df["Reliablity"] = rely_rank

result = normal_fuel_df[["Style","Reliablity","Fuel Economy"]]
result

,Style,Reliablity,Fuel Economy
Civic,0.1168,0.3788,0.300885
Saturn,0.2458,0.2900,0.238938
Escort,0.0607,0.0742,0.212389
Clio,0.5766,0.2570,0.247788


# The dot product of the alternatives rank matrix and the criteria weight matrix gives us the result

In [24]:
final = pd.DataFrame(np.dot(result,criteria_rank_df),index = Alternatives, columns = ["Score"])
final.sort_values(by = "Score",ascending = False)

,Alternatives
Clio,0.358020
Civic,0.285559
Saturn,0.269644
Escort,0.086745


# The winner has the higest Score which is the Clio